# Import Necessary Libraries

In [1]:
import numpy as np
import os
import mne
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, SimpleRNN
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GroupKFold
import tensorflow as tf
from tensorflow.keras import layers, models
from scipy.interpolate import interp1d

# Define X and Y

In [2]:
# Load the saved features from the .npz file
data = np.load('combined_dwt.npz') 

# Access the concatenated features from the .npz file
X_dwt = data['X_dwt']

# Close the loaded file
data.close()

#print the merged array
X_dwt

array([[[-1.90864583e-06, -1.01319508e-06, -1.25947210e-06, ...,
         -2.17607348e-07, -3.60696991e-07,  4.04866575e-07],
        [-4.16825421e-05, -4.20260902e-05, -4.24717352e-05, ...,
         -2.03555431e-07, -3.33857556e-07,  3.57200206e-07],
        [-1.90472102e-05, -1.92407524e-05, -1.70617167e-05, ...,
         -4.33967060e-08, -5.20276540e-08, -3.12188367e-08],
        ...,
        [ 6.94002021e-06,  6.58408463e-06,  6.57031824e-06, ...,
          8.69173749e-08, -1.77016500e-08,  3.94567816e-07],
        [ 2.41499888e-05,  2.47479670e-05,  2.44680713e-05, ...,
          1.14848042e-07,  1.76879477e-07, -1.72898476e-07],
        [-1.60015896e-06, -3.41169756e-06, -2.49139569e-06, ...,
          3.27513445e-07,  7.51374528e-07, -1.43600588e-06]],

       [[ 4.44703238e-05,  4.35599754e-05,  4.57796671e-05, ...,
         -8.22744389e-08, -2.28104073e-07,  4.71143282e-07],
        [ 5.61311547e-06,  4.87111596e-06,  6.55256693e-06, ...,
          1.00866868e-08,  1.83770888e

In [3]:
data = np.load('combined_labels.npz')

Y = data['Y']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

In [4]:
data = np.load('combined_groups.npz')

group = data['group']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

# Reshape Array

In [5]:
Y=Y
groups_array = np.hstack(group)

In [6]:
# Check the shape of the concatenated arrays
print(X_dwt.shape, Y.shape, groups_array.shape)

(73519, 20, 205) (73519,) (73519,)


In [7]:
# # Move the channel axis to the last dimension in the EEG data array
# epochs_array = np.moveaxis(X_merged_features, 1, 2)

# # Check the shape of the modified EEG data array
# epochs_array.shape

# Develop the Autoregression RNN Model

In [8]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout, BatchNormalization, Attention
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import numpy as np

# Define the improved CNN-LSTM hybrid model
def cnn_lstm_model(input_shape):

    learning_rate = 0.001
    dropout_rate = 0.3
    # Create a Sequential model
    model = Sequential()
    
    # Add Convolutional layers
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu', padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))
    
    model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))

    model.add(Conv1D(filters=256, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    # model.add(Dropout(dropout_rate))
    
    # Add Bidirectional LSTM layers
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    # model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    # model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(256)))
    # model.add(Dropout(dropout_rate))

    
    # Add Dense layers
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    optimizer=Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, matthews_corrcoef

# Define the evaluate_model function
def evaluate_model(model, features, labels):
    # Predict labels
    y_pred = model.predict(features)
    y_pred = np.round(y_pred).astype(int)
    
    # Calculate evaluation metrics
    accuracy = accuracy_score(labels, y_pred)
    precision = precision_score(labels, y_pred)
    recall = recall_score(labels, y_pred)
    f1 = f1_score(labels, y_pred)
    roc_auc = roc_auc_score(labels, y_pred)
    
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(labels, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    
    # Calculate kappa and MCC
    total = tp + tn + fp + fn
    observed_accuracy = (tp + tn) / total
    expected_accuracy = ((tp + fp) * (tp + fn) + (tn + fp) * (tn + fn)) / (total ** 2)
    kappa = (observed_accuracy - expected_accuracy) / (1 - expected_accuracy)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    
    return accuracy, precision, recall, f1, roc_auc, specificity, sensitivity, kappa, mcc

# Train the CNN LSTM Model

In [10]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger
import os
import csv

metrics_list = []

# Initialize confusion matrix sums
total_conf_matrix = [[0, 0], [0, 0]]

# Define the number of folds for cross-validation
n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Create a directory for TensorBoard logs
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X_dwt, Y), 1):
    print(f"Fold {fold}")

    # Split the data into training and validation sets
    X_train, X_val = X_dwt[train_index], X_dwt[val_index]
    y_train, y_val = Y[train_index], Y[val_index]

    # Standardize features using StandardScaler
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

    # Reshape the data for RNN input (samples, timesteps, features)
    timesteps = X_train_scaled.shape[1]
    features = X_train_scaled.shape[2]
    X_train_rnn = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, features))
    X_val_rnn = X_val_scaled.reshape((X_val_scaled.shape[0], timesteps, features))

    # Create an instance of the autoregression RNN model
    model = cnn_lstm_model((timesteps, features))

    # Set up TensorBoard callback
    tensorboard_callback = TensorBoard(log_dir=os.path.join(log_dir, f'fold_{fold}'))

    # Set up callbacks
    run_name = f"epoch_{fold}"
    checkpoint_filepath = f'{run_name}.keras'
    checkpoint_callback = ModelCheckpoint(
        checkpoint_filepath,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )

    csv_logger = CSVLogger(f'{run_name}.csv', append=True, separator=';')
    
    # Train the autoregression RNN model
    history = model.fit(X_train_rnn, y_train, epochs=200, batch_size=256, validation_data=(X_val_rnn, y_val), verbose=1, callbacks=[tensorboard_callback, checkpoint_callback, csv_logger])


    # Load the best model before evaluation
    best_model = load_model(checkpoint_filepath)

    # Evaluate client model
    accuracy, precision, recall, f1, sensitivity, specificity, roc_auc, kappa, mcc = evaluate_model(best_model, X_val_rnn, y_val)
    metrics_list.append({
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'ROC_AUC': roc_auc,
        'Kappa': kappa,
        'MCC': mcc
    })

Fold 1


C:\Users\khand\anaconda3\envs\mne\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 21s 50ms/step - accuracy: 0.8241 - loss: 0.4037 - val_accuracy: 0.8911 - val_loss: 0.2686
Epoch 2/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 12s 47ms/step - accuracy: 0.9017 - loss: 0.2467 - val_accuracy: 0.9181 - val_loss: 0.2082
Epoch 3/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 13s 49ms/step - accuracy: 0.9197 - loss: 0.1982 - val_accuracy: 0.9206 - val_loss: 0.2122
Epoch 4/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 13s 49ms/step - accuracy: 0.9323 - loss: 0.1726 - val_accuracy: 0.9297 - val_loss: 0.1772
Epoch 5/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 13s 49ms/step - accuracy: 0.9382 - loss: 0.1577 - val_accuracy: 0.9388 - val_loss: 0.1608
Epoch 6/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 12s 48ms/step - accuracy: 0.9435 - loss: 0.1414 - val_accuracy: 0.9279 - val_loss: 0.1790
Epoch 7/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 12s 48ms/step - accuracy: 0.9474 - loss: 0.1356 - val_accuracy: 0.9346 - val_loss: 0.1616
Epoch 8/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 13s 49ms/step - accuracy: 0.9516 - loss: 0

C:\Users\khand\anaconda3\envs\mne\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 36s 82ms/step - accuracy: 0.8206 - loss: 0.4000 - val_accuracy: 0.8983 - val_loss: 0.2505
Epoch 2/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 21s 83ms/step - accuracy: 0.8937 - loss: 0.2572 - val_accuracy: 0.9166 - val_loss: 0.2003
Epoch 3/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 20s 78ms/step - accuracy: 0.9194 - loss: 0.1996 - val_accuracy: 0.9242 - val_loss: 0.1809
Epoch 4/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 20s 77ms/step - accuracy: 0.9285 - loss: 0.1768 - val_accuracy: 0.9215 - val_loss: 0.1827
Epoch 5/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 20s 78ms/step - accuracy: 0.9375 - loss: 0.1586 - val_accuracy: 0.9276 - val_loss: 0.1747
Epoch 6/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 20s 79ms/step - accuracy: 0.9453 - loss: 0.1427 - val_accuracy: 0.9359 - val_loss: 0.1655
Epoch 7/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 20s 76ms/step - accuracy: 0.9482 - loss: 0.1337 - val_accuracy: 0.9291 - val_loss: 0.1783
Epoch 8/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 20s 79ms/step - accuracy: 0.9513 - loss: 0

...

Epoch 1/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 36s 103ms/step - accuracy: 0.8176 - loss: 0.4057 - val_accuracy: 0.8906 - val_loss: 0.2723
Epoch 2/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 26s 99ms/step - accuracy: 0.9022 - loss: 0.2457 - val_accuracy: 0.9117 - val_loss: 0.2196
Epoch 3/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 26s 100ms/step - accuracy: 0.9207 - loss: 0.1988 - val_accuracy: 0.9249 - val_loss: 0.1922
Epoch 4/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 26s 100ms/step - accuracy: 0.9305 - loss: 0.1738 - val_accuracy: 0.9211 - val_loss: 0.1893
Epoch 5/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 26s 101ms/step - accuracy: 0.9383 - loss: 0.1560 - val_accuracy: 0.9267 - val_loss: 0.1764
Epoch 6/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 26s 101ms/step - accuracy: 0.9436 - loss: 0.1436 - val_accuracy: 0.9253 - val_loss: 0.1742
Epoch 7/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 26s 101ms/step - accuracy: 0.9474 - loss: 0.1333 - val_accuracy: 0.9327 - val_loss: 0.1623
Epoch 8/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 26s 101ms/step - accuracy: 0.9522 - 

C:\Users\khand\anaconda3\envs\mne\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 41s 111ms/step - accuracy: 0.8200 - loss: 0.4080 - val_accuracy: 0.8851 - val_loss: 0.2736
Epoch 2/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 28s 107ms/step - accuracy: 0.8962 - loss: 0.2552 - val_accuracy: 0.9030 - val_loss: 0.2267
Epoch 3/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 28s 108ms/step - accuracy: 0.9193 - loss: 0.2059 - val_accuracy: 0.9120 - val_loss: 0.2159
Epoch 4/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 28s 108ms/step - accuracy: 0.9306 - loss: 0.1751 - val_accuracy: 0.9238 - val_loss: 0.1876
Epoch 5/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 28s 109ms/step - accuracy: 0.9368 - loss: 0.1572 - val_accuracy: 0.9245 - val_loss: 0.1935
Epoch 6/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 28s 108ms/step - accuracy: 0.9420 - loss: 0.1471 - val_accuracy: 0.9297 - val_loss: 0.1752
Epoch 7/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 28s 108ms/step - accuracy: 0.9478 - loss: 0.1347 - val_accuracy: 0.9238 - val_loss: 0.1898
Epoch 8/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 28s 109ms/step - accuracy: 0.9509 -

C:\Users\khand\anaconda3\envs\mne\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 45s 138ms/step - accuracy: 0.8143 - loss: 0.4143 - val_accuracy: 0.8904 - val_loss: 0.2796
Epoch 2/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 34s 130ms/step - accuracy: 0.9006 - loss: 0.2495 - val_accuracy: 0.8959 - val_loss: 0.2347
Epoch 3/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 34s 130ms/step - accuracy: 0.9202 - loss: 0.1993 - val_accuracy: 0.9163 - val_loss: 0.2069
Epoch 4/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 34s 130ms/step - accuracy: 0.9317 - loss: 0.1721 - val_accuracy: 0.9223 - val_loss: 0.1925
Epoch 5/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 34s 131ms/step - accuracy: 0.9396 - loss: 0.1531 - val_accuracy: 0.9295 - val_loss: 0.1892
Epoch 6/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 34s 130ms/step - accuracy: 0.9447 - loss: 0.1415 - val_accuracy: 0.9249 - val_loss: 0.1859
Epoch 7/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 34s 131ms/step - accuracy: 0.9472 - loss: 0.1345 - val_accuracy: 0.9342 - val_loss: 0.1712
Epoch 8/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 33s 129ms/step - accuracy: 0.9519 -

In [11]:
import pandas as pd
metrics_df = pd.DataFrame(metrics_list)

In [12]:
metrics_df.round(4)

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9665,0.9563,0.9821,0.9690,0.9654,0.9487,0.9821,0.9327,0.9331
1,0.9661,0.9606,0.9765,0.9685,0.9654,0.9543,0.9765,0.9319,0.9320
2,0.9668,0.9618,0.9765,0.9691,0.9661,0.9557,0.9765,0.9333,0.9334
3,0.9676,0.9605,0.9796,0.9699,0.9668,0.9540,0.9796,0.9349,0.9351
4,0.9674,0.9623,0.9770,0.9696,0.9667,0.9563,0.9770,0.9344,0.9345
5,0.9627,0.9519,0.9796,0.9655,0.9615,0.9435,0.9796,0.9250,0.9254
6,0.9656,0.9551,0.9816,0.9682,0.9645,0.9473,0.9816,0.9307,0.9312
7,0.9669,0.9644,0.9740,0.9691,0.9665,0.9589,0.9740,0.9336,0.9336
8,0.9675,0.9637,0.9758,0.9697,0.9669,0.9581,0.9758,0.9346,0.9347
9,0.9646,0.9607,0.9735,0.9670,0.9640,0.9546,0.9735,0.9289,0.9290


In [13]:
metrics_df.round(4).to_csv('DWT_COMBINED_CNN_LSTM.csv', index = False)

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
result = pd.read_csv("DWT_COMBINED_CNN_LSTM.csv")
result

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9665,0.9563,0.9821,0.9690,0.9654,0.9487,0.9821,0.9327,0.9331
1,0.9661,0.9606,0.9765,0.9685,0.9654,0.9543,0.9765,0.9319,0.9320
2,0.9668,0.9618,0.9765,0.9691,0.9661,0.9557,0.9765,0.9333,0.9334
3,0.9676,0.9605,0.9796,0.9699,0.9668,0.9540,0.9796,0.9349,0.9351
4,0.9674,0.9623,0.9770,0.9696,0.9667,0.9563,0.9770,0.9344,0.9345
5,0.9627,0.9519,0.9796,0.9655,0.9615,0.9435,0.9796,0.9250,0.9254
6,0.9656,0.9551,0.9816,0.9682,0.9645,0.9473,0.9816,0.9307,0.9312
7,0.9669,0.9644,0.9740,0.9691,0.9665,0.9589,0.9740,0.9336,0.9336
8,0.9675,0.9637,0.9758,0.9697,0.9669,0.9581,0.9758,0.9346,0.9347
9,0.9646,0.9607,0.9735,0.9670,0.9640,0.9546,0.9735,0.9289,0.9290


In [16]:
(result.mean()*100).round(2)

Accuracy       96.62
Precision      95.97
Recall         97.76
F1             96.86
Sensitivity    96.54
Specificity    95.31
ROC_AUC        97.76
Kappa          93.20
MCC            93.22
dtype: float64

In [17]:
# import csv
# for i in range (1, 11):
#     df = pd.read_csv(f'epoch_{i}.csv')
#     [col] = df.columns
#     split_data = df[col].str.split(';')
#     csv_file = (f'epoch_{i}.csv')
#     # Prepare CSV file and write headers
#     with open(csv_file, mode='w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(['Epoch', 'Training Accuracy', 'Training Loss', 'Validation Accuracy', 'Validation Loss'])
#     for j in range (0, 200):
#         with open(csv_file, mode='a', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerow([split_data[j][0], split_data[j][1], split_data[j][2], split_data[j][3], split_data[j][4]])